In [1]:
from tqdm import tqdm
import requests
import pandas as pd
import numpy as np

from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem, DataStructs

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPool1D

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

/home/milosevicn/.local/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2022-10-18 12:51:50.187388: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
models = dict()
cols = dict()

for model in ["ke1", "ke2", "ke3"]:
    models[model] = keras.models.load_model(f"models/skec/nn_{model}.h5")
    cols[model] = [model]

2022-10-18 12:51:52.803568: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-10-18 12:51:52.804437: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-10-18 12:51:53.263919: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-18 12:51:53.264182: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.815GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-10-18 12:51:53.264223: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-10-18 12:51:53.267714: I tensorflow/stream_executor/platf

In [3]:
test_data = [
    "data_tests/Chemical List EDSP21LIST2-2021-12-28.csv",
    "data_tests/Chemical List EDSPUOC-2022-06-16.csv",
    "data_tests/Chemical List OSHA-2021-12-28.csv",
    "data_tests/Chemical List UBAPMT-2021-12-28.csv",
    "data_tests/Chemical List EDSP21LIST1-2021-12-28.csv"    
]

In [4]:
def inference(row):
    try:
        smiles = str(row["smiles"])
        mol = Chem.MolFromSmiles(smiles)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2) # TODO: can replace morgan with other fingerprint!
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, arr)
        
        for model in ["ke1", "ke2", "ke3"]:
            # conv:
            arr = arr.reshape(-1, 2048, 1)
            out = list(models[model].predict(arr).round()[0])
            # fc:
            # out = list(models[model].predict(arr.reshape(1, -1)).round()[0])
            
            # print(out)
            
            for i, col in enumerate(cols[model]):
                row[col] = int(out[i])
        
    except Exception as e:
        # print(e)
        print("Bad smiles string? For:", row["PREFERRED NAME"])
        
    return row

In [ ]:
for test in test_data:    
    df = pd.read_csv(test)
    for col_list in cols.values():
        for col in col_list:
            df[col] = "unknown"

    df = df.apply(lambda x: inference(x), axis=1)
    
    df.to_csv(test.replace("data_tests/", "inference_results/skec/"), index=False)

Bad smiles string? For: Polychlorinated biphenyls


2022-10-18 12:52:16.638923: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-10-18 12:52:16.639509: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3192000000 Hz
2022-10-18 12:52:16.710827: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-10-18 12:52:16.853218: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
RDKit ERROR: [12:52:33] SMILES Parse Error: syntax error while parsing: ['Rgp'][N+]1(CC)CCOCC1.CCOS(=O)(=O)[O-]
[12:52:33] SMILES Parse Error: syntax error while parsing: ['Rgp'][N+]1(CC)CCOCC1.CCOS(=O)(=O)[O-]
RDKit ERROR: [12:52:33] SMILES Parse Error: Failed parsing SMILES '['Rgp'][N+]1(CC)CCOCC1.CCOS(=O)(=O)[O-]' for input: '['Rgp'][N+]1(CC)CCOCC1.CCOS(=O)(=O)[O-]'
[12:52:33] SMILES Parse Error: Failed parsing SMILES '['Rgp'][N+]1(CC)

Bad smiles string? For: Onium compounds, morpholinium, 4-ethyl-4-soya alkyl, Et sulfates


RDKit ERROR: [12:52:42] SMILES Parse Error: syntax error while parsing: [X]C(C)C.c1ccc(cc1)c2ccccc2
[12:52:42] SMILES Parse Error: syntax error while parsing: [X]C(C)C.c1ccc(cc1)c2ccccc2
RDKit ERROR: [12:52:42] SMILES Parse Error: Failed parsing SMILES '[X]C(C)C.c1ccc(cc1)c2ccccc2' for input: '[X]C(C)C.c1ccc(cc1)c2ccccc2'
[12:52:42] SMILES Parse Error: Failed parsing SMILES '[X]C(C)C.c1ccc(cc1)c2ccccc2' for input: '[X]C(C)C.c1ccc(cc1)c2ccccc2'


Bad smiles string? For: Isopropylbiphenyl


...

Bad smiles string? For: Dinitrotoluene


RDKit ERROR: [13:02:14] Explicit valence for atom # 4 C, 5, is greater than permitted
[13:02:14] Explicit valence for atom # 4 C, 5, is greater than permitted


Bad smiles string? For: (Methylcyclopentadienyl)tricarbonylmanganese


RDKit ERROR: [13:02:14] SMILES Parse Error: syntax error while parsing: [X]C(C)C.CC(C)c1ccccc1
[13:02:14] SMILES Parse Error: syntax error while parsing: [X]C(C)C.CC(C)c1ccccc1
RDKit ERROR: [13:02:14] SMILES Parse Error: Failed parsing SMILES '[X]C(C)C.CC(C)c1ccccc1' for input: '[X]C(C)C.CC(C)c1ccccc1'
[13:02:14] SMILES Parse Error: Failed parsing SMILES '[X]C(C)C.CC(C)c1ccccc1' for input: '[X]C(C)C.CC(C)c1ccccc1'


Bad smiles string? For: Diisopropylbenzene
